In [15]:
import os
import numpy as np

from pysc2.lib import actions
from pysc2.lib import features

In [16]:
PROJECT_DIR = os.path.abspath('../')
PARSED_DIR = os.path.join(PROJECT_DIR, 'parsed')
assert all([os.path.isdir(PROJECT_DIR), os.path.isdir(PARSED_DIR)])

In [17]:
def load_features(replay_dir, feature_type='screen'):
    """Load parsed features from .npz file format."""
    if feature_type == 'screen':
        filepath = os.path.join(replay_dir, 'ScreenFeatures.npz')
    elif feature_type == 'minimap':
        filepath = os.path.join(replay_dir, 'MinimapFeatures.npz')
    elif feature_type == 'flat':
        filepath = os.path.join(replay_dir, 'FlatFeatures.npz')
        raise NotImplementedError
    else:
        raise ValueError
    
    with np.load(filepath) as fp:
        name2feature = {k: v for k, v in fp.items()}
    
    return name2feature

In [18]:
def human_readable_size(size, precision=2):
    suffixes = ['B','KB','MB','GB','TB']
    suffix_idx = 0
    while size > 1024 and suffix_idx < 4:
        suffix_idx += 1     # increment the index of the suffix
        size = size / 1024.0  # apply the division
    return "%.*f%s" % (precision, size, suffixes[suffix_idx])

In [49]:
for replay_dir in os.listdir(PARSED_DIR):
    
    # Load features
    print('Replay Name: %s' % replay_dir)
    replay_dir = os.path.join(PARSED_DIR, replay_dir)
    screen_features = load_features(replay_dir, 'screen')    # dict
    minimap_features = load_features(replay_dir, 'minimap')  # dict
    
    # Split keys and values into separate tuples
    screen_names, screen_features = zip(*screen_features.items())
    minimap_names, minimap_features = zip(*minimap_features.items())
    
    # Stack features along the channel axis (1)
    screen_features = np.stack(screen_features, axis=1)
    minimap_features = np.stack(minimap_features, axis=1)
    
    print('-' * 70)
    print(
        'SCREEN  | shape: {} | size: {}'.format(
            screen_features.shape, human_readable_size(screen_features.nbytes)
        )
    )
    # Check order consistency with pysc2.lib.features.SCREEN_FEATURES
    for sc_name, sc_spec in zip(screen_names, features.SCREEN_FEATURES):
        assert sc_name == sc_spec.name
        name_ = sc_spec.name
        type_ = sc_spec.type.__str__().split('.')[-1]
        scale_ = sc_spec.scale
        print('Name:{:>25} | Type:{:>15} ({})'.format(name_, type_, scale_))
    
    print('-' * 70)
    print(
        'MINIMAP | shape: {}  | size: {}'.format(
            minimap_features.shape, human_readable_size(minimap_features.nbytes)
        )
    )
    
    # Check order consistency with pysc2.lib.features.MINIMAP_FEATURES
    for mn_name, mn_spec in zip(minimap_names, features.MINIMAP_FEATURES):
        assert mn_name == mn_spec.name
        name_ = mn_spec.name
        type_ = mn_spec.type.__str__().split('.')[-1]
        scale_ = mn_spec.scale
        print('Name:{:>25} | Type:{:>15} ({})'.format(name_, type_, scale_))
    
    print('=' * 70)
    break

Replay Name: 000779266779913ff54c46c6ecb601143a646acab761ffc3de1e58eadcabaa34
----------------------------------------------------------------------
SCREEN  | shape: (1551, 17, 64, 64) | size: 411.98MB
Name:               height_map | Type:         SCALAR (256)
Name:           visibility_map | Type:    CATEGORICAL (4)
Name:                    creep | Type:    CATEGORICAL (2)
Name:                    power | Type:    CATEGORICAL (2)
Name:                player_id | Type:    CATEGORICAL (17)
Name:          player_relative | Type:    CATEGORICAL (5)
Name:                unit_type | Type:    CATEGORICAL (1914)
Name:                 selected | Type:    CATEGORICAL (2)
Name:          unit_hit_points | Type:         SCALAR (1600)
Name:    unit_hit_points_ratio | Type:         SCALAR (256)
Name:              unit_energy | Type:         SCALAR (1000)
Name:        unit_energy_ratio | Type:         SCALAR (256)
Name:             unit_shields | Type:         SCALAR (1000)
Name:       unit_shields_